In [ ]:
!pip install selenium webdriver-manager

In [13]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import UnexpectedAlertPresentException
import re
import time


options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 1. 브라우저 설정
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get("https://gcloud.csi.go.kr/cmq/main.do")
    print("\n" + "="*70)
    print(" [Work Instruction]")
    print(" 1. Log in and open the detailed page.")
    print(" 2. After the page is fully loaded, press Enter here.")
    print("="*70)
    input(">> Press Enter to start Excel saving: ")

    # 엑셀에 담을 데이터 리스트
    excel_data = []

    print("\n" + "="*70)
    print(" [Data Extraction and Excel Processing Started]")
    print("="*70)

    # ---------------------------------------------------------
    # [1] 하단부: 현재 완벽하게 작동하는 로직
    # ---------------------------------------------------------
    
    # 1-1. 인적 사항
    print("\n[1. Collecting Personnel Information...]")
    for s in ["pick_user_text", "seal_user_text", "attnd_user_text"]:
        try:
            row = driver.find_element(By.XPATH, f"//th[@data-name='{s}']/..")
            val = row.text.strip().replace('\n', ' | ')
            print(f"> {val}")
            excel_data.append(["1.Personnel", s, val])
        except: continue

    # 1-2. 공사개요
    print("\n[2. Collecting Construction Summary...]")
    c_fields = {"Construction Name": "bizNm", "Site Address": "addr", "Start Date": "bgngDt", "Completion Date": "cmplPlanDt"}
    for label, eid in c_fields.items():
        try:
            el = driver.find_element(By.ID, eid)
            val = el.get_attribute('value') or el.get_attribute('innerText').strip() or el.text.strip()
            print(f"> {label} : {val}")
            excel_data.append(["2.Summary", label, val])
        except: continue

    # 1-3. 기타 항목
    print("\n[3. Collecting Other Items...]")
    for label in ["성과 이용 목적", "발주자", "시공자", "국가중요시설 여부"]:
        try:
            val = driver.find_element(By.XPATH, f"//th[contains(text(), '{label}')]/following-sibling::td").text.strip()
            print(f"> {label} : {val}")
            excel_data.append(["3.Others", label, val])
        except: continue

    # ---------------------------------------------------------
    # [2] 상단부: 과거 성공 로직 적용 (테이블 스캔)
    # ---------------------------------------------------------
    print("\n[4. Collecting Top Section Information...]")
    try:
        top_container = driver.find_element(By.ID, "rqstViewAjax")
        top_tables = top_container.find_elements(By.TAG_NAME, "table")
        
        for table in top_tables:
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                text = row.get_attribute('innerText').strip().replace('\n', ' | ')
                if text:
                    print(f"  {text}")
                    excel_data.append(["4.TopInfo", "Details", text])
    except Exception as e:
        print(f"> Reference for Top Section: {e}")

    # ---------------------------------------------------------
    # [3] 엑셀 파일 저장
    # ---------------------------------------------------------
    df = pd.DataFrame(excel_data, columns=["Category", "Item", "Content"])
    file_name = "Quality_Test_Result.xlsx"
    df.to_excel(file_name, index=False)

    print("\n" + "="*70)
    print(f" Success! All data saved to '{file_name}'.")
    print(" Check the Excel file in your folder.")
    print("="*70)

except Exception as e:
    print(f"\n[Error] {e}")



 [Work Instruction]
 1. Log in and open the detailed page.
 2. After the page is fully loaded, press Enter here.

 [Data Extraction and Excel Processing Started]

[1. Collecting Personnel Information...]
> 채취자 소속 | 현대엔지니어링 주식회사 | 담당업무 | 품질관리 | 성명 | 정광훈 (서명 완료)

[2. Collecting Construction Summary...]
> Construction Name : 부천대장 공종주택지구 조성공사 2공구
> Site Address : 경기도 부천시 오정구 벌말로 43
> Start Date : 2024-04-25

[3. Collecting Other Items...]
> 성과 이용 목적 : 품질시험·검사 대행
> 발주자 : 한국토지주택공사 인천지역본부 (담당자: 최기찬)
> 시공자 : 현대엔지니어링 주식회사
> 국가중요시설 여부 : 해당없음

[4. Collecting Top Section Information...]
  의뢰번호	RQ-2025-244241	접수번호	AC-2025-330146	최종진행상태	시험·검사 진행 확정
  품질검사 대행기관	한국건설품질시험원 주식회사
  시험실명 및 주소	한국건설품질시험원 (주소:경기도 고양시 일산서구 송포백송길 47 (덕이동))	시험실 연락처	031-946-5518
  의뢰기관	현대엔지니어링 주식회사	의뢰인 | (의뢰인 번호)	정광훈 | (02--)	의뢰일시	2025-12-30 10:30:46
  입찰공고 마감일시	2025-12-30 12:00	의뢰 확인자	심억	입찰 공고 일시	2025-12-30 10:38:32
  접수일시	2025-12-30 13:08:27	시험·검사진행 | 확정일시	2025-12-30 13:08:27	성적서 발급 | 예정일자	2026-02-09 이력
  봉인명	원심력철근콘크리트관(D900)